# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [147]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import joblib

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [148]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')

In [149]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [150]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=21, stratify=y_train)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [ ]:
svm = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', probability=True, random_state=21)
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_valid)

accuracy = accuracy_score(y_valid, svm_y_pred)
precision = precision_score(y_valid, svm_y_pred, average='weighted')
recall = recall_score(y_valid, svm_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.87778
precision is 0.88162
recall is 0.87778


In [152]:
dt = DecisionTreeClassifier(class_weight='balanced', criterion= 'gini', max_depth= 22, random_state=21)
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_valid)

accuracy = accuracy_score(y_valid, dt_y_pred)
precision = precision_score(y_valid, dt_y_pred, average='weighted')
recall = recall_score(y_valid, dt_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.86667
precision is 0.86984
recall is 0.86667


In [153]:
rfc = RandomForestClassifier(class_weight=None, criterion='gini', max_depth=28, n_estimators=50, random_state=21)
rfc.fit(X_train, y_train)
rfc_y_pred = rfc.predict(X_valid)

accuracy = accuracy_score(y_valid, rfc_y_pred)
precision = precision_score(y_valid, rfc_y_pred, average='weighted')
recall = recall_score(y_valid, rfc_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.89259
precision is 0.89361
recall is 0.89259


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [154]:
voting_clf = VotingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)])
voting_clf.fit(X_train, y_train)
voting_y_pred = voting_clf.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.89630
precision is 0.89605
recall is 0.89630


In [155]:
voting_clf = VotingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)], voting='soft')
voting_clf.fit(X_train, y_train)
voting_y_pred = voting_clf.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.88148
precision is 0.88418
recall is 0.88148


In [156]:
voting_clf = VotingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)], voting='hard')
voting_clf.fit(X_train, y_train)
voting_y_pred = voting_clf.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.89630
precision is 0.89605
recall is 0.89630


In [157]:
voting_clf = VotingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)], voting='hard')
voting_clf.fit(X_train, y_train)
voting_y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, voting_y_pred)
precision = precision_score(y_test, voting_y_pred, average='weighted')
recall = recall_score(y_test, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.91124
precision is 0.91206
recall is 0.91124


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [158]:
bag = BaggingClassifier(estimator=svm, n_estimators=50, random_state=21)
bag.fit(X_train, y_train)
voting_y_pred = bag.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.88148
precision is 0.89035
recall is 0.88148


In [159]:
bag = BaggingClassifier(estimator=svm, n_estimators=48, random_state=21)
bag.fit(X_train, y_train)
voting_y_pred = bag.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.87778
precision is 0.88741
recall is 0.87778


In [160]:
bag = BaggingClassifier(estimator=svm, n_estimators=52, random_state=21)
bag.fit(X_train, y_train)
voting_y_pred = bag.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.88519
precision is 0.89396
recall is 0.88519


In [161]:
bag = BaggingClassifier(estimator=svm, n_estimators=52, random_state=21)
bag.fit(X_train, y_train)
voting_y_pred = bag.predict(X_test)

accuracy = accuracy_score(y_test, voting_y_pred)
precision = precision_score(y_test, voting_y_pred, average='weighted')
recall = recall_score(y_test, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.87870
precision is 0.88360
recall is 0.87870


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [162]:
def cv_gen(n):
    return StratifiedKFold(n_splits=n, shuffle=True, random_state=21)

In [163]:
stack = StackingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)], cv=cv_gen(5), passthrough=False)
stack.fit(X_train, y_train)
voting_y_pred = stack.predict(X_valid)

accuracy = accuracy_score(y_valid, voting_y_pred)
precision = precision_score(y_valid, voting_y_pred, average='weighted')
recall = recall_score(y_valid, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.91481
precision is 0.91781
recall is 0.91481


In [164]:
stack = StackingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)], cv=cv_gen(5), passthrough=False, final_estimator=LogisticRegression(solver='liblinear', multi_class='auto'))
stack.fit(X_train, y_train)
voting_y_pred = stack.predict(X_test)

accuracy = accuracy_score(y_test, voting_y_pred)
precision = precision_score(y_test, voting_y_pred, average='weighted')
recall = recall_score(y_test, voting_y_pred, average='weighted')

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.90237
precision is 0.90522
recall is 0.90237


## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [165]:
model = VotingClassifier(estimators=[
    ('svm', svm), ('dt', dt), ('rfc', rfc)], voting='hard')
model.fit(X_train, y_train)
voting_y_pred = model.predict(X_test)
error_df = X_test.copy()
error_df['predict'] = voting_y_pred
error_df['error'] = (voting_y_pred != y_test)
total = len(X_test)

In [166]:
weekday_error = error_df[error_df['error'] == True].groupby('predict').count()['error']/total * 100
weekday_error_max = weekday_error.idxmax()
print(f"weekday with the most errors: {weekday_error_max}")
print(f"{weekday_error[weekday_error_max]:.2f}%")

weekday with the most errors: 5
2.66%


In [167]:
labname_columns = [col for col in error_df.columns if col.startswith('labname_')]
labname_error_counts = error_df[error_df['error'] == True][labname_columns].sum()/total * 100
labname_with_most_errors = labname_error_counts.idxmax()
print(f'Labname with the most errors: {labname_with_most_errors}')
print(f"{labname_error_counts[labname_with_most_errors]:.2f}%")

Labname with the most errors: labname_project1
2.66%


In [168]:
uid_columns = [col for col in error_df.columns if col.startswith('uid_user_')]
uid_error = error_df[error_df['error'] == True][uid_columns].sum()/total * 100
uid_with_most_errors = uid_error.idxmax()
print(f'Labname with the most errors: {uid_with_most_errors}')
print(f"{uid_error[uid_with_most_errors]:.2f}%")

Labname with the most errors: uid_user_2
1.48%


In [ ]:
joblib.dump(model, '../data/best_model_ex03.pkl')

['../data/best_model_ex03.pkl']

: 